# In-Context Learning
* Be careful when doing this, as the cost is very high.

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@develop

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.47


In [3]:
from dotenv import load_dotenv

# .env ファイルを読み込む
load_dotenv(override=True)

True

In [4]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import ICLClassifier

In [12]:
seed = 3655
#seed = 3656
#seed = 3657
train_data = 300

file_name = f'pseudodata_{seed}_{train_data}_train.csv'

i = 200

df = pd.read_csv(f'../data/pseudodata/{file_name}').head(i)
x_train = df.drop('target', axis=1)
y_train = df['target']
print(len(x_train))

200


In [13]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iclm = ICLClassifier(llm_model = llm_model)

### Make prompt

In [14]:
icl_prompt = iclm.fit(x_train, y_train)

Number of input tokens:5710


In [15]:
print(icl_prompt)


        Predict the probability value by observing all of the following conditions.
        ・Output must be numeric only.
        ・Do not output any text.
        ・Predicting probability values as finely as possible increases overall accuracy.
        ・Please refer to the following data to estimate the probability that "target" will be 1. The rightmost column with a value of 0 or 1 is 'target'.
        ------------------
        1.88,-2.14,1.56,-1.52,1.0
-0.02,0.35,0.16,-0.02,0.0
0.29,-0.47,0.18,-0.22,1.0
0.51,-0.35,0.54,-0.44,0.0
1.03,-1.07,0.91,-0.85,1.0
1.01,-1.11,0.85,-0.82,0.0
0.27,0.67,0.72,-0.32,1.0
-0.89,-0.44,-1.49,0.87,0.0
1.07,-1.07,0.96,-0.88,1.0
-0.72,0.39,-0.82,0.63,0.0
0.42,0.52,0.86,-0.44,1.0
0.03,-2.06,-1.02,0.19,0.0
1.49,-1.79,1.19,-1.2,1.0
-0.46,-0.88,-1.1,0.52,0.0
0.69,1.29,1.63,-0.77,1.0
-0.25,-1.28,-1.01,0.37,0.0
0.1,0.5,0.4,-0.15,1.0
-0.04,-0.53,-0.33,0.1,0.0
1.16,-1.07,1.09,-0.96,1.0
0.03,0.43,0.26,-0.07,0.0
0.97,-0.83,0.94,-0.81,1.0
-0.88,1.42,-0.52,0.67,0.0
0

### Prediction

In [16]:
file_name = f'pseudodata_{seed}_{train_data}_test.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

In [17]:
y_proba = iclm.predict(x_test)

y_pred = (y_proba > 0.5).astype(int)

output = pd.DataFrame(y_proba, columns=['pred'])
data_name = 'pseudodata'
output.to_csv(f'../data/icl_output/pred_{data_name}_{seed}.csv')

100%|██████████| 300/300 [09:57<00:00,  1.99s/it]


In [21]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(accuracy, 4)}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {round(precision, 4)}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {round(recall, 4)}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {round(f1, 4)}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {round(roc_auc, 4)}')

Accuracy: 0.8767
Precision: 0.8294
Recall: 0.9463
F1 score: 0.884
ROC-AUC: 0.9395


In [8]:
columns = 4
train_data = 300

file_name = f'pseudodata_{columns}f_{train_data}_test.csv'

df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

prefix = 1
y_proba = pd.read_csv(f'../data/icl_output/pred.csv')
y_proba = y_proba.drop('Unnamed: 0', axis=1)
y_pred = (y_proba > 0.5).astype(int)

In [9]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

ValueError: Found input variables with inconsistent numbers of samples: [700, 300]